<a href="https://colab.research.google.com/github/increpare/tatoeba_toki_pona_spellcheck/blob/main/tatoeba_toki_pona_spellcheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# by jan Inkepa - https://github.com/increpare/tatoeba_toki_pona_spellcheck/blob/main/tatoeba_toki_pona_spellcheck.ipynb

import os
import urllib.request

print("downloading toki pona sentences from tatoeba")
urllib.request.urlretrieve('https://downloads.tatoeba.org/exports/per_language/toki/toki_sentences.tsv.bz2', 'toki_sentences.tsv.bz2')

!rm -rf /content/toki_sentences.tsv

print("decompressing data")
!bunzip2 /content/toki_sentences.tsv.bz2
print("done")


downloading toki pona sentences from tatoeba
decompressing data
done


In [13]:
import pandas as pd
df = pd.read_csv("/content/toki_sentences.tsv",names=["k","v"],delimiter="\t")
words = ["a","akesi","ala","ali","alasa","ale","anpa","ante","anu","awen","e","en","esun","ijo","ike","ilo","insa","jaki","jan","jelo","jo","kala","kalama","kama","kasi","ken","kepeken","kili","kin","kipisi","kiwen","ko","kon","kule","kulupu","kute","la","lape","laso","lawa","leko","len","lete","li","lili","linja","lipu","loje","lon","luka","lukin","lupa","ma","mama","mani","meli","mi","mije","moku","moli","monsi","monsuta","mu","mun","musi","mute","namako","nanpa","nasa","nasin","nena","ni","nimi","noka","o","oko","olin","ona","open","pakala","pali","palisa","pan","pana","pi","pilin","pimeja","pini","pipi","poka","poki","pona","pu","sama","seli","selo","seme","sewi","sijelo","sike","sin","sina","sinpin","sitelen","sona","soweli","suli","suno","supa","suwi","tan","taso","tawa","telo","tenpo","toki","tomo","tonsi","tu","unpa","uta","utala","walo","wan","waso","wawa","weka","wile"]

In [28]:
import re

def validate(w):
  if w[0].isupper():
    return w.lower() not in words
  return w in words

#checks for "mi/sina" li sentences, and roughly for sentences with an "e" but without a "li"/"o"/"mi"/"sina"
def validate_sentence(s,tokens,index):
  if (tokens[0]=="mi" or tokens[0]=="sina") and tokens[1]=="li":
    print(s) 
    print("\thttp://tatoeba.org/eng/sentences/show/"+str(index))
    print("\t\tERROR: mi\sina li")
  if "li" not in tokens and "e" in tokens and "o" not in tokens:
    if "mi" not in tokens and "sina" not in tokens:      
      print(s)
      print("\thttps://tatoeba.org/eng/sentences/show/"+str(index))
      print("\t\tERROR: 'e' without 'li'/'mi'/'sina'/'o'")
  if bool(re.search(r"\bla,? (mi|sina) li\b",s)):
    print(s)
    print("\thttps://tatoeba.org/eng/sentences/show/"+str(index))
    print("\t\tERROR: mi\sina li")
  lastchar = s[len(s)-1]
  if lastchar.isalnum():
    print(s)
    print("\thttp://tatoeba.org/eng/sentences/show/"+str(index))
    print("\t\tERROR: Sentence doesn't end with punctuation.")

freq={}
links={}
id_table={}

for index, row in df.iterrows():
  import re
  id_table[index]=row
  tokens = re.findall(r'\w+',row['v'])
  validate_sentence(row['v'],tokens,index)
  for w in tokens:
    if not validate(w):
      if w not in freq:
        freq[w]=0
        links[w]=[]
      freq[w]=freq[w]+1
      links[w].append(index)
      break

jan lili mute li musi lon ma kasi pi ma tomo
	http://tatoeba.org/eng/sentences/show/642739
		ERROR: Sentence doesn't end with punctuation.
tenpo suno kama la seli li pona la mi tawa tan insa
	http://tatoeba.org/eng/sentences/show/667636
		ERROR: Sentence doesn't end with punctuation.
pilin pi kasi kule Lila li lon ma kasi pi poka tomo mute
	http://tatoeba.org/eng/sentences/show/740208
		ERROR: Sentence doesn't end with punctuation.
jan mute li olin e jan ni pi kalama pona sin ala
	http://tatoeba.org/eng/sentences/show/1419652
		ERROR: Sentence doesn't end with punctuation.
sina tawa lili taso la sina li kama moli.
	https://tatoeba.org/eng/sentences/show/1722700
		ERROR: mi\sina li
jan Losi li musi uta ala
	http://tatoeba.org/eng/sentences/show/1725107
		ERROR: Sentence doesn't end with punctuation.
mi en meli mi li wan
	http://tatoeba.org/eng/sentences/show/3002378
		ERROR: Sentence doesn't end with punctuation.
kute tan kama sona. kute tan ala toki
	http://tatoeba.org/eng/sentences/sh

In [ ]:
d_view = [ (v,k) for k,v in freq.items() ]
d_view.sort(reverse=True) # natively sort tuples by first element

for k,w in d_view:
    print("%s: %d" % (w,k))
    for id in links[w]:
      print("\t"+id_table[id]['v'])
      print("\t\thttps://tatoeba.org/eng/sentences/show/"+str(id))